In [ ]:
# importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from cmath import sqrt
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# IRELAND

In [ ]:
irelandagri_consumer = pd.read_csv('ireland/FAOSTAT_data_5-1-2022.csv')
irelandagri_consumer.head()

In [ ]:
irelandagri_consumer.tail()

In [ ]:
irelandagri_consumer.describe()

In [ ]:
irelandagri_consumer.info()

In [ ]:
irelandagri_consumer.shape

In [ ]:
irelandagri_crops = pd.read_csv('ireland/FAOSTAT_data_5-1-2022 (2).csv')
irelandagri_crops.head()

In [ ]:
irelandagri_crops.tail()

In [ ]:
irelandagri_crops.describe()

In [ ]:
irelandagri_crops.info()

In [ ]:
irelandagri_crops.shape

#Now that the duplicates are gone we are going to merge using an inner join
#The base data set is co2 so with the inner join if hter where more countries in the other they will appear the same.
# Dataframe of world regions new dataframe for merge
#NB CO2 is the left dataframe
#so merging df_gm without the duplicates notice where using small country because we are joining on the co2 country column
#Anything that does not exist in the 1st table Co2 is discarded from the second table inner join



In [ ]:
df_irelandagri_consumer = irelandagri_consumer[['Year','Value','Months']].drop_duplicates()

In [ ]:
 irishdata = pd.merge(irelandagri_crops,df_irelandagri_consumer,left_on="Year",right_on="Year", how ="inner")

In [ ]:
irishdata.head()

In [ ]:
irishdata.tail()

In [ ]:
irishdata.shape

In [ ]:
irishdata.describe()

In [ ]:
irishdata.info()

In [ ]:
#dropping the large Country feature no need for 2 of the same and we will keep the lowercase one
#irishdata= irishdata.drop("Note","Flag","Area_x","Area_y")
#irishdata.drop(['Note','Flag','Area_y','Domain Code','Year Code','Area Code','Item Code','Months Code'], axis=1, inplace=True)

In [ ]:
#change all year type to ints to ensure there are no strings for the years greater than 2009 and this covers the years for this data 2010 to 2020
#The second 3rd line we we ensure that year is ints again sometimes after sorting or merging they can turn into strings

df_irishdata = irishdata[irishdata['Year'].astype('int64') > 2009]
df_irishdata = df_irishdata.sort_values(by=['Area', 'Year'])
df_irishdata['Year'] = df_irishdata['Year'].astype('int64')
df_irishdata.head()

In [ ]:
df_irishdata.shape

In [ ]:
#Rename our columns to be more descriptive names for some and ensure all of the uppercase ones are 
#lowercase to keep in line with naming conventions
df_irishdata = df_irishdata.rename(columns={'Domain':'crops_domain',
                       'Year':'year',
                       'Item':'crops_type',
                       'Value_x':'crop_value',
                       'Element':'crops_action',
                       'Unit':'crop_measurement',
                       'Area':'country',
                       'Months':'months',
                       'Value_y':'consumer_value'})

In [ ]:
df_irishdata.head()

In [ ]:
#Reorder Columns

irishcolumns = ["year","country","months","crop_measurement","crops_type","crops_action","crop_value","consumer_value"]
df_irishdata = df_irishdata.reindex(columns = irishcolumns)

# Missing Data

In [ ]:
# check all columns for NAN values if any

df_irishdata.isna().any()

In [ ]:
# check the sum of null records

df_irishdata.isnull().sum()


In [ ]:
#Checking dataframe for missing data
%matplotlib inline
plt.figure(figsize=(10, 5))
colourmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(df_irishdata.isnull(), cmap=colourmap)

In [ ]:
#To zoom in on the missing data
missing_data = df_irishdata.iloc[:, 6:7]
plt.figure(figsize=(20, 10))
colourmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(missing_data.isnull(), cmap=colourmap)

In [ ]:
import plotly.express as px
fig = px.box(df_irishdata, y='consumer_value')
fig.show()

In [ ]:
import seaborn as sns
#
# Box plot
#
sns.boxplot(df_irishdata.crop_value)


In [ ]:
#
# Distribution plot
#
sns.distplot(df_irishdata.consumer_value)

In [ ]:
#
# Distribution plot
#
sns.distplot(df_irishdata.crop_value)

In [ ]:
##check rows where dependent variable is equal to zero
df_irishdata.loc[df_irishdata['crop_value']==0]

In [ ]:
#We can see for the crop value column there are over 11,268 rows with the value 0 I we want to find the mean value of the column
print(df_irishdata['crop_value'].mean()) 

In [ ]:
#For all rows with zero for the column crop_value replace them with the average value listed above and below.
df_irishdata['crop_value'] = df_irishdata['crop_value'].replace(0, 384548.5637279597) 

In [ ]:
#We have 5,184 rows for the column crop_value that are null I am going to replace them too with the mean average value for that feature
# Replace NaNs in column crop_value with the
# mean of values in the same column
mean_value=df_irishdata['crop_value'].mean()
df_irishdata['crop_value'].fillna(value=mean_value, inplace=True)
print('Updated Dataframe:')
print(df_irishdata)

In [ ]:
# check the sum of null records again it should be 0 this time

df_irishdata.isnull().sum()

In [ ]:
plt.figure(figsize = (16,4))
sns.boxplot(x= df_irishdata['crop_value'])
plt.show()

In [ ]:
plt.figure(figsize = (16,4))
sns.boxplot(x= df_irishdata['consumer_value'])
plt.show()

In [ ]:
##check cardinality

v_cardinality = df_irishdata.nunique()

print(v_cardinality)

In [ ]:
##plot cardinality using a barplot

df_irishdata.nunique().plot.bar(figsize=(12,6))

plt.ylabel('Number of unique categories')

plt.xlabel('Variables')

plt.title('Cardinality')

In [ ]:
#checking the shape and size
print("Shape of the dataset is : ",df_irishdata.shape)
print("Size of the dataset is : ",df_irishdata.size)

In [ ]:
#Box plot visualisation to depict the crop price from 2010 to 2020
plt.figure(figsize = (16,4))
sns.boxplot(y='crop_value', x='year',data=df_irishdata)
plt.show()

In [ ]:
df_irishdata.head()

In [ ]:
#create two arrays to compare coefficients
consumervalue=np.array(df_irishdata["consumer_value"])
cropvalue=np.array(df_irishdata["crop_value"])

In [ ]:
#how to work out coefficients between gdp and 
np.corrcoef (consumervalue, cropvalue)

# The result tells us that there is a high correlation between consumer value and cropvalue 

Lets use a scatter plot to plot the results

In [ ]:
import plotly.express as px

In [ ]:
#Were creating two variables for our min and max values for the slider
xmin,xmax=min(df_irishdata["crop_value"]), max(df_irishdata["crop_value"])
ymin,ymax=min(df_irishdata["consumer_value"]), max(df_irishdata["consumer_value"])

In [ ]:
px.scatter(df_irishdata, x="crop_value", y="consumer_value",
animation_frame="months",
size="year", color="crops_type", hover_name="country",
size_max = 50,
range_x=[0.1, 50], range_y=[100,60000],
log_x=True, log_y=True)
fig.show()

In [ ]:
import plotly.express as px

In [ ]:
#Were creating two variables for our min and max values for the slider
xmin,xmax=min(df_irishdata["year"]), max(df_irishdata["year"])
ymin,ymax=min(df_irishdata["crop_value"]), max(df_irishdata["crop_value"])

In [ ]:
#Using year as the animation frame, and as we go through each year the values will change want to group it by country
#The color will be the region each region will have a different color, using hover for the country to give us info
#Split up the regions by using facet hight 1580 in pixels, lots of large and small values in gdp so were going to use log
#to be able to see all the values, we can do the same with y.  Size is used for size of points
fig= px.scatter(df_irishdata, x="year", y="crop_value", animation_frame ="year",
               animation_group="months",color="crops_type", hover_name="months", 
               facet_col="crops_action", width=1580, height=400, log_x=True, size_max=10000,
               range_x=[xmin,xmax],range_y=[ymin,ymax])
               

In [ ]:
fig.show()

# Brazil

In [ ]:
brazilagri_crops = pd.read_csv('brazil/FAOSTAT_brazil_5-1-2022(4).csv')
brazilagri_crops.head()

In [ ]:
brazilagri_crops.tail()

In [ ]:
brazilagri_crops.describe()

In [ ]:
brazilagri_crops.info()

In [ ]:
brazilagri_crops.shape

In [ ]:
brazilagri_consumer = pd.read_csv('brazil/FAOSTATbrazil_data_5-1-2022.csv')
brazilagri_consumer.head()

In [ ]:
brazilagri_consumer.tail()

In [ ]:
brazilagri_consumer.describe()

In [ ]:
brazilagri_consumer.info()

In [ ]:
brazilagri_consumer.shape

In [ ]:
df_brazilagri_consumer = brazilagri_consumer[['Year','Value','Months']].drop_duplicates()

In [ ]:
brazildata = pd.merge(brazilagri_crops,df_brazilagri_consumer,left_on="Year",right_on="Year", how ="inner")

In [ ]:
brazildata.head()

In [ ]:
brazildata.tail()

In [ ]:
brazildata.describe()

In [ ]:
brazildata.info()

In [ ]:
brazildata.shape

In [ ]:
#change all year type to ints to ensure there are no strings for the years greater than 2009 and this covers the years for this data 2010 to 2020
#The second 3rd line we we ensure that year is ints again sometimes after sorting or merging they can turn into strings

df_brazildata = brazildata[brazildata['Year'].astype('int64') > 2009]
df_brazildata = df_brazildata.sort_values(by=['Area', 'Year'])
df_brazildata['Year'] = df_brazildata['Year'].astype('int64')
df_brazildata.head()

In [ ]:
#dropping the large Country feature no need for 2 of the same and we will keep the lowercase one
#irishdata= irishdata.drop("Note","Flag","Area_x","Area_y")
#brazildata.drop(['Note','Flag','Area_x','Area_y','Domain Code','Year Code','Area Code','Item Code','Months Code','Unit_x'], axis=1, inplace=True)

In [ ]:
df_brazildata.shape

In [ ]:
#Rename our columns to be more descriptive names for some and ensure all of the uppercase ones are 
#lowercase to keep in line with naming conventions
df_brazildata = df_brazildata.rename(columns={'Domain':'crops_domain',
                       'Year':'year',
                       'Item':'crops_type',
                       'Value_x':'crop_value',
                       'Element':'crops_action',
                       'Unit':'crop_measurement',
                       'Area':'country',
                       'Months':'months',
                       'Value_y':'consumer_value'})

In [ ]:
df_brazildata.head()

In [ ]:
#Reorder Columns

brazilcolumns = ["year","country","months","crop_measurement","crops_type","crops_action","crop_value","consumer_value"]
df_brazildata = df_brazildata.reindex(columns = brazilcolumns)

# Missing Data

In [ ]:
# check all columns for NAN values if any

df_brazildata.isna().any()

In [ ]:
# check the sum of null records
df_brazildata.isnull().sum()

In [ ]:
#Checking dataframe for missing data
%matplotlib inline
plt.figure(figsize=(10, 5))
colourmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(df_brazildata.isnull(), cmap=colourmap)

In [ ]:
#To zoom in on the missing data
missing_data = df_brazildata.iloc[:, 6:7]
plt.figure(figsize=(20, 10))
colourmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=True)
sns.heatmap(missing_data.isnull(), cmap=colourmap)

In [ ]:
import plotly.express as px
fig = px.box(df_brazildata, y='consumer_value')
fig.show()

In [ ]:
import seaborn as sns
#
# Box plot
#
sns.boxplot(df_brazildata.crop_value)

In [ ]:
sns.boxplot(x=df_brazildata['consumer_value'])

In [ ]:
#
# Distribution plot
#
sns.distplot(df_brazildata.crop_value)

In [ ]:
##check rows where dependent variable is equal to zero
df_brazildata.loc[df_brazildata['crop_value']==0]

In [ ]:
#We can see for the crop value column there are over 612 rows with the value 0 I we want to find the mean value of the column
print(df_brazildata['crop_value'].mean()) 

In [ ]:
#For all rows with zero for the column crop_value replace them with the average value listed above and below.
df_brazildata['crop_value'] = df_brazildata['crop_value'].replace(0, 6041652.271239063) 

In [ ]:
#We have 2.376 rows for the column crop_value that are null I am going to replace them too with the mean average value for that feature
# Replace NaNs in column crop_value with the
# mean of values in the same column
mean_value=df_brazildata['crop_value'].mean()
df_brazildata['crop_value'].fillna(value=mean_value, inplace=True)
print('Updated Dataframe:')
print(df_brazildata)

In [ ]:
# check the sum of null records again it should be 0 this time

df_brazildata.isnull().sum()

In [ ]:
plt.figure(figsize = (16,4))
sns.boxplot(x= df_brazildata['crop_value'])
plt.show()

In [ ]:
##check cardinality

v_cardinality = df_brazildata.nunique()

print(v_cardinality)

In [ ]:
##check cardinality
v_cardinality = df_brazildata.nunique()
print(v_cardinality)


In [ ]:
##plot cardinality using a barplot

df_brazildata.nunique().plot.bar(figsize=(12,6))

plt.ylabel('Number of unique categories')

plt.xlabel('Variables')

plt.title('Cardinality')

In [ ]:
#checking the shape and size
print("Shape of the dataset is : ",df_brazildata.shape)
print("Size of the dataset is : ",df_brazildata.size)

In [ ]:
#Box plot visualisation to depict the crop price from 2010 to 2020
plt.figure(figsize = (16,4))
sns.boxplot(x='year', y='crop_value', data=df_brazildata)
plt.show()

In [ ]:
df_brazildata.head()

In [ ]:
#create two arrays to compare coefficients
consumervalue=np.array(df_brazildata["consumer_value"])
cropvalue=np.array(df_brazildata["crop_value"])

In [ ]:
#how to work out coefficients between gdp and 
np.corrcoef (consumervalue, cropvalue)

# The result tells us that there is a high correlation between consumer value and cropvalue

Below is a scatterplot to plot the results

In [ ]:
import plotly.express as px

In [ ]:
#Were creating two variables for our min and max values for the slider
xmin,xmax=min(df_brazildata["crop_value"]), max(df_brazildata["crop_value"])
ymin,ymax=min(df_brazildata["consumer_value"]), max(df_brazildata["consumer_value"])

In [ ]:
px.scatter(df_brazildata, x="crop_value", y="consumer_value",
animation_frame="year",
animation_group="country", color="crops_type", hover_name="country",
size_max = 50,
range_x=[0.1, 50], range_y=[100,60000],
log_x=True, log_y=True)
fig.show()

In [ ]:
import plotly.express as px

In [ ]:
#Were creating two variables for our min and max values for the slider
xmin,xmax=min(df_brazildata["crop_value"]), max(df_brazildata["crop_value"])
ymin,ymax=min(df_brazildata["consumer_value"]), max(df_brazildata["consumer_value"])

In [ ]:
#Using year as the animation frame, and as we go through each year the values will change want to group it by country
#The color will be the region each region will have a different color, using hover for the country to give us info
#Split up the regions by using facet hight 1580 in pixels, lots of large and small values in gdp so were going to use log
#to be able to see all the values, we can do the same with y.  Size is used for size of points
fig= px.scatter(df_brazildata, x="crop_value", y="consumer_value", animation_frame ="year",
               animation_group="country",color="crops_type", hover_name="months", 
               facet_col="months", width=1580, height=400, log_x=True, size_max=45,
               range_x=[xmin,xmax],range_y=[ymin,ymax])

In [ ]:
fig.show()

In [ ]:
df_brazildata.head()

# EU

In [ ]:
irelandagri_consumer = pd.read_csv('eu\FAOSTAT_data_5-9-2022.csv')
irelandagri_consumer.head()

In [ ]:
irelandagri_consumer = pd.read_csv('eu\FAOSTAT_data_5-9-2022(1).csv')
irelandagri_consumer.head()

In [ ]:
FAOSTAT_data_5-9-2022(1)